In [35]:
import pandas as pd
import numpy as np
import os
import glob
import calendar
from datetime import datetime


# Definig a function to converting month txt to number
def month_text_number_conversion(month_text):
    
    try:
        # Converting abbreviated month text to number (e.g. 'Jan' to 1)
        month_num = datetime.strptime(month_name.upper(), '%b').month
        return month_num
    
    except:
        # Converting full month text to number (e.g. 'January' to 1)
        month_num = datetime.strptime(month_name.upper(), '%B').month
        return month_num


# Looping through all the folder to process the raw CPA water level data to timeseries data
for folder in glob.glob(r"H:\MJI\SHWL_SLWL\CPA\CPA_Data\**"):
    
    # Setting xls folder path
    folder_path = r"{}\*.xls".format(folder)
    
    # folder path to create a folder path list to get the folder name
    folder_path_list = folder.split(os.sep)
    
    # Defining a blank dataframe to store the yearly timeseries data
    ts_df = pd.DataFrame()
    
    try:
        # Looping through all the files of the folder
        for file in glob.glob(folder_path):

            file_path = file

            # Storing the xls file to get the file information (e.g. sheet name)
            xls = pd.ExcelFile(file_path)

            # Separating the xls file path to get the year from the file name
            file_path_list = file_path.split(os.sep)
            year = int(file_path_list[-1][-8:-4])

            # Defining a blank dataframe to store the yearly timeseries data
            yearly_df = pd.DataFrame()

            # Looping through all sheet in the excel file
            for month_name in xls.sheet_names:

                # Reading the sheet one by one from the excel file
                df = pd.read_excel(file_path, sheet_name = month_name)

                # Converting the month text to month number using the "month_text_number_conversion" function
                month_num = month_text_number_conversion(month_name)

                # Calculating the total number of days in a month using month range function
                month_days = calendar.monthrange(year, month_num)[1]

                # Setting sixth row as columns header and renaming the "Time" column as "Day"
                df.rename(columns=df.iloc[5], inplace =True)
                df.rename(columns = {'Time': 'Day'}, inplace = True)

                # Subsetting the required rows and column for timeseries preparation
                raw_df = df.iloc[7:7+month_days,:25]

                # Converting column data to montly timeseris @hourly data using pandas "melt" method
                monthly_df = raw_df.melt(id_vars='Day',value_vars=raw_df.iloc[:,1:], var_name ='Time', value_name ='WL')

                # Adding month & year column and creating date from day,month,year 
                # and also converting time column to time format
                monthly_df[['Month','Year']] = [month_num, year]
                monthly_df['Date'] = pd.to_datetime(dict(year=monthly_df['Year'], month=monthly_df['Month'], 
                                                      day=monthly_df['Day']), format='%Y-%m-%d')
                monthly_df['Time'] = pd.to_datetime(monthly_df['Time'], format='%H:%M:%S')

                # Converting "Date" and "Time" column to time string format
                monthly_df['Date']  = monthly_df['Date'].dt.strftime('%d/%m/%Y')
                monthly_df['Time']  = monthly_df['Time'].dt.strftime('%H:%M:%S')

                # Creating a datetime column by merging date and time string column
                monthly_df['Date/Time'] = pd.to_datetime(monthly_df['Date'] + ' ' + monthly_df['Time'], 
                                                         format ='%d/%m/%Y %H:%M:%S')

                # Sorting the dataframe by date/time
                monthly_df.sort_values(by='Date/Time', ascending=True, inplace=True)

                # Updating the montly timeseries to the yearly dataframe
                yearly_df = pd.concat([yearly_df, monthly_df])

            # Updating the updated yearly timeseries to the total timeseries dataframe
            ts_df = pd.concat([ts_df, yearly_df])

        # Selecting the required column from the total timeseies dataframe to create the required timeseries dataframe
        wl_ts_df = ts_df[['Date/Time', 'WL']]

        # Saving the processed water level timeseries dataframe
        wl_ts_df.to_excel(r"H:\MJI\SHWL_SLWL\CPA\Processed_Data\{}.xlsx".format(file_path_list[-2]),
                          float_format="%.3f", index = None)
    
    except:
        print(folder_path_list[-1])